# Cleaning Profile Data
The profiles are composed by consumption and production data. The data is real and anonymized, which will provided a more realistic representation of the behaviour of the systems in this study. The data is composed by:
- Consumption data: 69 time stamped data with the duration of one year.
- PV Production data: 10 time stamped data with the duration of one year.
- Wind Production data: 20 time stamped data with the duration of one year.


## PV Production Data
First, it is necessary to import the data and explore its structure.

In [24]:
import pandas as pd
import matplotlib.pyplot as plt

# import utils from thesis_package in the parent directory.
import sys
sys.path.append('..')
from thesis_package import utils as ut

Import the data into the python jupyter notebook.

In [25]:
# Get path to the parent folder.
# Import each data frame into a list.
def describe_data(data):
    print('Number of df: ', len(data))
    print('Head of df: \n', data[0].head())
    # Describre the dataframes.
    for i, df  in enumerate(data):
        print('File: ', i)
        print(df.describe())
        print('___________________________')
data_path = '..\data\\raw\profile\PV\IPMA (2 years 2020-2021)\\'
pv_data = ut.get_csv_from_folder(data_path)
describe_data(pv_data)

Number of df:  10
Head of df: 
                   date  value   T   R
0  2020-01-01 00:00:00    0.0 NaN NaN
1  2020-01-01 00:15:00    0.0 NaN NaN
2  2020-01-01 00:30:00    0.0 NaN NaN
3  2020-01-01 00:45:00    0.0 NaN NaN
4  2020-01-01 01:00:00    0.0 NaN NaN
File:  0
              value             T             R
count  70176.000000  70153.000000  70153.000000
mean     383.269572    288.969642    747.648987
std      559.559322      3.871006    739.292716
min        0.000000    275.930000      0.000000
25%        0.000000    286.407333     87.480646
50%        0.000000    288.729667    535.380093
75%      706.000000    291.471250   1217.935185
max     1909.000000    304.526000   2838.527778
___________________________
File:  1
              value             T             R
count  34238.000000  70153.000000  70153.000000
mean    1670.871838    290.069160    757.128517
std     2393.891953      5.158342    743.964051
min        0.000000    273.663000      0.000000
25%        0.000000   

#### Raw PV Data Description:
The full dataset is composed by a total of 10 time stamped data with the duration of one to two years. The the data is composed by the following features:
- Time: Time stamp of the data.
- PV Production: PV production power in kW.
- Temperature: Temperature in degrees K.
- Irradiance: Irradiance in W/m2.

### Analysing Missing Values
The data contains missing values. Since the production values are the focous for this study, it is necessary to deal with the missing values. In order to better understand the distribution of the missing values, the following questions will be answared:
- Number of missing values?
- Percentage of missing values overall?
- Percentage of missing values per year?
- Percentage of missing values per month?

In [26]:
def add_year_month_to_df(data):
    for df in data:
        df['year'] = df.date.apply(lambda x: x.split('-')[0])
        df['month'] =  df.date.apply(lambda x: x.split('-')[1])
add_year_month_to_df(pv_data)

In [27]:
# Get percentage of missing data per file per feature.
def get_percentage_of_per_feature(i, df):
    df_result = pd.DataFrame()
    for feature in df.columns:
        if feature != 'date' and feature != 'year' and feature != 'month':
            df_result.loc[i, feature] = 100 * df[feature].isnull().sum() / len(df)
    return df_result
def get_percentage_of_missing_data_per_file_per_feature(data):
    result = pd.DataFrame()
    for i, df in enumerate(data):
        result  = pd.concat([result, get_percentage_of_per_feature(i, df)], axis=0)
    return result
# Get percentage of missing data per file per year and/or month per feature.
def get_percentage_of_missing_values_per(data, time_frame=['year']):
    result = []
    for df in data:
        df_result = pd.DataFrame()
        for feature in df.columns:
                    if feature != 'date' and feature != 'year' and feature != 'month':
                        df_result[feature] = df.groupby(time_frame)[feature].apply(lambda x: 100 * x.isnull().sum() / len(x))
        result.append(df_result)
    return result

In [28]:
percentage_of_missing_data_per_file_per_feature = get_percentage_of_missing_data_per_file_per_feature(pv_data)
percentage_of_missing_values_per_year = get_percentage_of_missing_values_per(pv_data, time_frame=['year'])
percentage_of_missing_values_per_year_per_month = get_percentage_of_missing_values_per(pv_data, time_frame=['year', 'month'])

Analysing the percentage of missing data per file. we have that:

In [29]:
percentage_of_missing_data_per_file_per_feature

,value,T,R
0,0.000000,0.032775,0.032775
1,51.203592,0.017103,0.017103
2,22.116440,0.017103,0.017103
3,2.771603,0.032775,0.032775
4,2.771603,0.032775,0.032775
5,0.000000,0.032775,0.032775
6,0.000000,0.032775,0.032775
7,0.000000,0.032775,0.032775
8,1.335214,0.032775,0.032775
9,0.000000,0.032775,0.032775


From the `percentage_of_missing_data_per_file_per_feature` dataframe we can note that the file 1  and 2 contain around 50% and 20% of missing values, respectively. 

Anakysing the percentage of missing data per file per year, we have that:

In [30]:
percentage_of_missing_values_per_year

[      value         T         R
 year                           
 2020    0.0  0.034153  0.034153
 2021    0.0  0.031393  0.031393,
            value         T         R
 year                                
 2020    2.555783  0.034153  0.034153
 2021  100.000000  0.000000  0.000000,
           value         T         R
 year                               
 2020   0.000000  0.034153  0.034153
 2021  44.300437  0.000000  0.000000,
          value         T         R
 year                              
 2020  5.535633  0.034153  0.034153
 2021  0.000000  0.031393  0.031393,
          value         T         R
 year                              
 2020  5.535633  0.034153  0.034153
 2021  0.000000  0.031393  0.031393,
       value         T         R
 year                           
 2020    0.0  0.034153  0.034153
 2021    0.0  0.031393  0.031393,
       value         T         R
 year                           
 2020    0.0  0.034153  0.034153
 2021    0.0  0.031393  0.031393,
       va

From the `percentage_of_missing_data_per_file_per_year_per_feature` dataframe we can note the year 2020 has a smaller percentage of missing values, so it is a good choice for obtaining preliminary results. So it is suitable for the cleaning process.

Now we can check the distribution of missing values per month, per year.

In [31]:
percentage_of_missing_values_per_year_per_month

[            value         T         R
 year month                           
 2020 01       0.0  0.403226  0.403226
      02       0.0  0.000000  0.000000
      03       0.0  0.000000  0.000000
      04       0.0  0.000000  0.000000
      05       0.0  0.000000  0.000000
      06       0.0  0.000000  0.000000
      07       0.0  0.000000  0.000000
      08       0.0  0.000000  0.000000
      09       0.0  0.000000  0.000000
      10       0.0  0.000000  0.000000
      11       0.0  0.000000  0.000000
      12       0.0  0.000000  0.000000
 2021 01       0.0  0.000000  0.000000
      02       0.0  0.000000  0.000000
      03       0.0  0.000000  0.000000
      04       0.0  0.000000  0.000000
      05       0.0  0.000000  0.000000
      06       0.0  0.000000  0.000000
      07       0.0  0.000000  0.000000
      08       0.0  0.000000  0.000000
      09       0.0  0.000000  0.000000
      10       0.0  0.000000  0.000000
      11       0.0  0.000000  0.000000
      12       0.0  0.369

From the `percentage_of_missing_data_per_file_per_year_per_month_per_feature` dataframe we can note that the month of february has a a null percentage of missing values, so it is a good choice for obtaining preliminary results.

## Wind Production Data

In [32]:
data_path = '..\data\\raw\profile\Wind\IPMA (2 years 2020-2021)\\'
wind_data = ut.get_csv_from_folder(data_path)
describe_data(wind_data)

Number of df:  20
Head of df: 
                   date  value   T   R  wind_speed  wind_direction
0  2020-01-01 00:00:00    0.0 NaN NaN         NaN             NaN
1  2020-01-01 00:15:00    0.0 NaN NaN         NaN             NaN
2  2020-01-01 00:30:00    0.0 NaN NaN         NaN             NaN
3  2020-01-01 00:45:00    0.0 NaN NaN         NaN             NaN
4  2020-01-01 01:00:00    0.0 NaN NaN         NaN             NaN
File:  0
              value             T             R    wind_speed  wind_direction
count  63524.000000  70153.000000  70153.000000  70153.000000    70153.000000
mean    3001.348932    289.893199    798.823422      7.403072      239.481862
std     2874.484190      3.707198    760.917081      3.207411      113.411583
min        0.000000    275.922000      0.000000      0.122154        0.058645
25%      520.000000    287.412000     98.727006      4.861206      127.899805
50%     2040.000000    289.783667    610.015278      7.260239      288.147924
75%     5000.0000

#### Raw Wind Data Description:
The full dataset is composed by a total of 20 time stamped data with the duration of one to two years. The the data is composed by the following features:
- Time: Time stamp of the data.
- Wind Production: Wind production power in kW.
- Temperature: Temperature in degrees K.
- Irradiance: Irradiance in W/m2.
- Wind Speed: Wind speed in m/s.
- Wind Direction: Wind direction in degrees.

In [33]:
add_year_month_to_df(wind_data)
percentage_of_missing_data_per_file_per_feature = get_percentage_of_missing_data_per_file_per_feature(wind_data)
percentage_of_missing_values_per_year = get_percentage_of_missing_values_per(wind_data, time_frame=['year'])
percentage_of_missing_values_per_year_per_month = get_percentage_of_missing_values_per(wind_data, time_frame=['year', 'month'])

In [34]:
percentage_of_missing_data_per_file_per_feature

,value,T,R,wind_speed,wind_direction
0,9.479024,0.032775,0.032775,0.032775,0.032775
1,0.000000,0.032775,0.032775,0.032775,0.032775
2,0.000000,0.032775,0.032775,0.032775,0.032775
3,0.000000,0.032775,0.032775,0.032775,0.032775
4,5.159884,0.032775,0.032775,0.032775,0.032775
5,0.000000,0.032775,0.032775,0.032775,0.032775
6,0.000000,0.032775,0.032775,0.032775,0.032775
7,0.000000,0.032775,0.032775,0.032775,0.032775
8,0.000000,0.032775,0.032775,0.032775,0.032775
9,0.000000,0.032775,0.032775,0.032775,0.032775


From the `percentage_of_missing_data_per_file_per_feature` dataframe we can note that there is not any file with more than 15% of any feature missing.

Now we check if the month of february has a null percentage of missing values.

In [35]:
percentage_of_missing_values_per_year_per_month

[                 value         T         R  wind_speed  wind_direction
 year month                                                            
 2020 01       0.000000  0.403226  0.403226    0.403226        0.403226
      02       0.000000  0.000000  0.000000    0.000000        0.000000
      03       0.000000  0.000000  0.000000    0.000000        0.000000
      04       0.000000  0.000000  0.000000    0.000000        0.000000
      05       0.000000  0.000000  0.000000    0.000000        0.000000
      06       0.000000  0.000000  0.000000    0.000000        0.000000
      07       0.000000  0.000000  0.000000    0.000000        0.000000
      08       0.000000  0.000000  0.000000    0.000000        0.000000
      09       0.000000  0.000000  0.000000    0.000000        0.000000
      10       0.000000  0.000000  0.000000    0.000000        0.000000
      11       0.000000  0.000000  0.000000    0.000000        0.000000
      12       0.000000  0.000000  0.000000    0.000000        0

From the output above, it is confirmed that indeed the month of february has a null percentage of missing values.

## Consumption Data

In [36]:
data_path = '..\data\\raw\profile\Conso\Clean Data\\'
conso_data = ut.get_csv_from_folder(data_path)
describe_data(conso_data)
add_year_month_to_df(conso_data)

Number of df:  64
Head of df: 
                   date     P    Q           T           R
0  2020-01-01 00:00:00  70.0  2.0  281.969500  405.454167
1  2020-01-01 00:15:00  72.0  2.0  281.924375  371.666319
2  2020-01-01 00:30:00  68.0  2.0  281.879250  337.878472
3  2020-01-01 00:45:00  70.0  6.0  281.834125  304.090625
4  2020-01-01 01:00:00  72.0  5.0  281.789000  270.302778
File:  0
                  P             Q             T             R
count  45569.000000  45569.000000  45569.000000  45569.000000
mean     122.579627     27.089359    287.147408    648.861610
std       53.927606     18.626930      5.445740    668.677534
min       22.000000     -3.000000    272.962000      0.000000
25%       79.000000     12.000000    283.646458     76.829062
50%      110.000000     22.000000    286.565833    447.426389
75%      158.000000     41.000000    290.329333   1001.572531
max      359.000000     98.000000    307.022000   2815.203704
___________________________
File:  1
                

### Raw Consumption Data Description:
The full dataset is composed by a total of 69 time stamped data with the duration of one year. The the data is composed by the following features:
- Time: Time stamp of the data.
- Active Power Consumption: Consumption in kW.
- Reactive Power Consumption: Reactive power consumption in kVAr.
- Temperature: Temperature in degrees K.
- Irradiance: Irradiance in W/m2.

In [37]:
percentage_of_missing_data_per_file_per_feature = get_percentage_of_missing_data_per_file_per_feature(conso_data)

In [38]:
percentage_of_missing_data_per_file_per_feature

,P,Q,T,R
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
...,...,...,...,...
59,0.0,0.0,0.0,0.0
60,0.0,0.0,0.0,0.0
61,0.0,0.0,0.0,0.0
62,0.0,0.0,0.0,0.0


From the `percentage_of_missing_data_per_file_per_feature` dataframe we do not have any missing data.

## Create Preliminary Results Dataset
In order to obtain preliminary results, the data from the month of Feburary of 2020 will be used.

In [39]:
# Get the datapoints from the dataframes from a given year and month.
pv_data
df = pv_data[0]  
def get_data_from_month_of_year(df, year, month):
    return df[(df.year == year) & (df.month == month)]	

In [40]:
# Get all the rows that belong go the year 2020 and month 02.